In [1]:
import sys
import math
import numpy as np

In [54]:
class point(object):
    def __init__(self, x, y, value=0):
        self.x = x
        self.y = y
        self.value = value
    def __eq__(self, another):
        return self.x == another.x and self.y == another.y
    def __hash__(self):
        return hash("{}-{}".format(self.x, self.y))
    def __repr__(self):
        return "{} {}".format(self.x, self.y)

def surroundings(p, map_grid, pallet_grid):
    search_list = []
    for x, y in [[p.x+1, p.y], [p.x, p.y+1], [p.x-1, p.y], [p.x, p.y-1]]:
        x_new = int(x % width)
        y_new = int(y % height)
        if map_grid[x_new][y_new] == " ":
            search_list.append(point(x_new, y_new, pallet_grid[x_new][y_new]))
    return search_list 

def bfs_search(p, map_grid, pallet_grid):
    d = [p]
    marked = set()
    while len(d) > 0:
        p_current = d.pop(0)
        if p_current.value > 0:
            return p_current
        d += [p for p in surroundings(p_current, map_grid, pallet_grid) if p not in marked]
        marked.add(p_current)

In [55]:
map_grid = [[" ", " ", "#", " ", " "],
           ["#", "#", "#", " ", "#"],
           [" ", " ", " ", " ", " "],
           ["#", "#", "#", "#", "#"]]

pallet_grid = [[0, 0, 0, 0, 0],
               [0, 0, 0, 1, 0],
               [0, 0, 1, 1, 0],
               [0, 0, 0, 0, 0]]

In [56]:
bfs_search(point(0, 0), map_grid, pallet_grid)

1 3

In [ ]:
width, height = [int(i) for i in input().split()]
map_grid = []
for i in range(height):
    row = input()  # one line of the grid: space " " is floor, pound "#" is wall
    map_grid.append([x for x in row])
map_grid = np.array(map_grid).transpose()

## Game Loop

In [ ]:
while True:
    my_score, opponent_score = [int(i) for i in input().split()]
    visible_pac_count = int(input())  # all your pacs and enemy pacs in sight
    my_pacs = []
    oppenent_pacs = []
    for i in range(visible_pac_count):
        inputs = input().split()
        pac_id = int(inputs[0])  # pac number (unique within a team)
        mine = inputs[1] != "0"  # true if this pac is yours
        x = int(inputs[2])  # position in the grid
        y = int(inputs[3])  # position in the grid
        type_id = inputs[4]  # unused in wood leagues
        speed_turns_left = int(inputs[5])  # unused in wood leagues
        ability_cooldown = int(inputs[6])  # unused in wood leagues
        if mine:
            my_pacs.append([x, y, pac_id])
        else:
            oppenent_pacs.append([x, y, pac_id])
    visible_pellet_count = int(input())  # all pellets in sight

    pallet_grid = np.zeros((width, height))
    for i in range(visible_pellet_count):
        # value: amount of points this pellet is worth
        x, y, value = [int(j) for j in input().split()]
        pallet_grid[x][y] = value

    # Write an action using print
    # To debug: print("Debug messages...", file=sys.stderr, flush=True)
    commands = []
    for pac in my_pacs:
        x, y, type_id = pac
        commands.append("MOVE {} {}".format(type_id, bfs_search(point(x, y), map_grid, pallet_grid)))

    print("|".join(commands))